<a href="https://colab.research.google.com/github/mag381/AI/blob/main/02_karaage_%E9%A1%94%E8%A1%A8%E6%83%85%E5%88%A4%E5%AE%9A_%E9%AB%98%E6%80%A7%E8%83%BD_%E3%82%AB%E3%83%A1%E3%83%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# カメラでの画像分類（性能改善版）


学習させたモデルを使ってカメラ画像に対して画像分類をします。

注：このノートブックはハードウェアアクセラレータをGPUよりNone(CPU)で実行した方がレスポンスが良い場合もあります。

## 事前準備

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
print(tf.__version__)

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
2.18.0


In [ ]:
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import numpy as np
import time

モデルとラベルをGoogle Driveからコピーします。

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp '/content/drive/My Drive/my_model_aug.h5' './my_model_aug.h5'
#!cp '/content/drive/My Drive/efficientnet_model.h5' './efficientnet_model.h5'
!cp '/content/drive/My Drive/labels.txt' './labels.txt'

In [ ]:
!ls -lh

total 91M
drwx------ 6 root root 4.0K May  4 08:31 drive
-rw------- 1 root root   20 May  4 08:32 labels.txt
-rw------- 1 root root  91M May  4 08:32 my_model_aug.h5
drwxr-xr-x 1 root root 4.0K Apr 30 13:37 sample_data


## AIモデルとラベルの読み込み

EfficientNetモデルは使わないので、ここはスキップします。<p>

EfficientNetモデルを読み込む場合は以下実行ください

In [ ]:
# keras_model = tf.keras.models.load_model("efficientnet_model.h5")

import tensorflow_hub as hub
keras_model = tf.keras.models.load_model('efficientnet_model.h5', custom_objects={'KerasLayer':hub.KerasLayer})
keras_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 1280)              4049564   
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 3843      
Total params: 4,053,407
Trainable params: 3,843
Non-trainable params: 4,049,564
_________________________________________________________________


参考： https://github.com/tensorflow/tensorflow/issues/26835

データ水増しして学習させたMNISTベースの自作モデルを読み込む場合は、以下実行ください。
（EfficientNetを試したい場合は、こちらはスキップしてください）。

In [ ]:
keras_model = tf.keras.models.load_model("my_model_aug.h5")
keras_model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)               │ (None, 64, 64, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_6 (Activation)       │ (None, 64, 64, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 62, 62, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_7 (Activation)       │ (None, 62, 62, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 31, 31, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 31, 31, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 61504)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 128)            │     7,872,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_8 (Activation)       │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 3)              │           387 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_9 (Activation)       │ (None, 3)              │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,892,421 (30.11 MB)

 Trainable params: 7,892,419 (30.11 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

ラベルを読み込みます

In [ ]:
labels = []
with open('./labels.txt','r') as f:
  for line in f:
    labels.append(line.rstrip())
print(labels)

NUM_CLASSES = len(labels)

['angry', 'happy', 'neutral']


## 画像分類デモの実行

Webカメラを使うJavascriptのコードをダウンロードします。

In [ ]:
!wget https://raw.githubusercontent.com/karaage0703/karaage-ai-book/master/util/colab_camera.py

--2025-05-04 08:33:57--  https://raw.githubusercontent.com/karaage0703/karaage-ai-book/master/util/colab_camera.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3036 (3.0K) [text/plain]
Saving to: ‘colab_camera.py’

colab_camera.py     100%[===================>]   2.96K  --.-KB/s    in 0s      

2025-05-04 08:33:57 (30.7 MB/s) - ‘colab_camera.py’ saved [3036/3036]



コードを読み込みます。

In [ ]:
import colab_camera

処理を行うコールバック関数を定義します。

In [ ]:
import IPython
from google.colab import output
import cv2
import numpy as np
from PIL import Image
from io import BytesIO
import base64

def run(img_str):
  #decode to image
  decimg = base64.b64decode(img_str.split(',')[1], validate=True)
  decimg = Image.open(BytesIO(decimg))
  decimg = np.array(decimg, dtype=np.uint8);
  decimg = cv2.cvtColor(decimg, cv2.COLOR_BGR2RGB)

  # image classify
  img = cv2.resize(decimg, (64, 64))
  img = img_to_array(img)
  img = img / 255.0
  img = img[np.newaxis]
  img = np.asarray(img)
  preds = keras_model.predict(img)
  result = np.argmax(preds[0])

  # draw output
  out_img = decimg
  cv2.putText(out_img, labels[result], (10,50), \
      cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 1, cv2.LINE_AA)

  #encode to string
  _, encimg = cv2.imencode(".jpg", out_img,
      [int(cv2.IMWRITE_JPEG_QUALITY), 80])
  img_str = encimg.tobytes()
  img_str = ("data:image/jpeg;base64," +
      base64.b64encode(img_str).decode('utf-8'))
  return IPython.display.JSON({'img_str': img_str})

output.register_callback('notebook.run', run)

デモを実施します。

In [ ]:
colab_camera.use_cam()

<IPython.core.display.Javascript object>

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 274ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━

# 参考リンク

- https://qiita.com/a2kiti/items/f32de4f51a31d609e5a5